In [8]:
#! pip install torch


In [9]:
#! pip install --upgrade pip

In [13]:
import scipy.io as sio
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MPIIFaceGazeDataset(Dataset):
    def __init__(self, mat_file_path, image_dir, transform=None):
        """
        mat_file_path: مسار ملف التعليقات (.mat)
        image_dir: مجلد الصور الأصلية
        """
        self.data = sio.loadmat(mat_file_path)
        self.image_dir = image_dir
        self.transform = transform
        
        # استخراج البيانات
        self.images = self.data['Data']['image'][0, 0]  # مسارات الصور
        self.gaze = self.data['Data']['gaze'][0, 0]      # متجهات النظر (3D)
        self.head_pose = self.data['Data']['pose'][0, 0] # وضعية الرأس
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # قراءة الصورة
        img_path = f"{self.image_dir}/{self.images[idx][0]}"
        image = Image.open(img_path).convert('RGB')
        
        # استخراج labels
        gaze_label = torch.tensor(self.gaze[idx], dtype=torch.float32)
        head_label = torch.tensor(self.head_pose[idx], dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
            
        return image, gaze_label, head_label

# استخدام:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

dataset = MPIIFaceGazeDataset('path/to/annotations.mat', 'path/to/images', transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/annotations.mat'

In [12]:
#! pip install scipy
#! pip install torchvision

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class GazeEstimationModel(nn.Module):
    def __init__(self, output_dim=3):
        super(GazeEstimationModel, self).__init__()
        # استخدام ResNet18 كـ Backbone
        self.backbone = models.resnet18(pretrained=True)
        
        # تعديل الطبقة الأخيرة لإخراج متجه 3D (اتجاه النظر)
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, output_dim)  # x, y, z للنظر
        )
        
    def forward(self, x):
        gaze = self.backbone(x)
        # تسوية المتجه ليكون وحدة طول (Unit Vector)
        gaze = F.normalize(gaze, p=2, dim=1)
        return gaze

# دوال التدريب:
def angular_loss(pred, target):
    """حساب الخطAngular بين متجهين"""
    cos_sim = F.cosine_similarity(pred, target)
    # تحويل cos similarity إلى زاوية بالراديان
    angle = torch.acos(torch.clamp(cos_sim, -1.0, 1.0))
    return torch.mean(angle)

# مثال على loop تدريب:
model = GazeEstimationModel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for images, gaze_gt, _ in loader:
    optimizer.zero_grad()
    gaze_pred = model(images)
    loss = angular_loss(gaze_pred, gaze_gt)
    loss.backward()
    optimizer.step()

c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


NameError: name 'loader' is not defined

In [15]:
def evaluate_gaze_model(model, dataloader, device='cuda'):
    model.eval()
    total_error = 0.0
    count = 0
    
    with torch.no_grad():
        for images, gaze_gt, _ in dataloader:
            images = images.to(device)
            gaze_gt = gaze_gt.to(device)
            
            gaze_pred = model(images)
            
            # حساب الخط الزاوي بالدرجات
            cos_sim = F.cosine_similarity(gaze_pred, gaze_gt)
            angles = torch.acos(torch.clamp(cos_sim, -1.0, 1.0)) * (180.0 / np.pi)
            
            total_error += torch.sum(angles).item()
            count += len(angles)
    
    mean_error = total_error / count
    print(f"Mean Angular Error: {mean_error:.2f} degrees")
    return mean_error